# Teleportación cuántica y codificación superdensa

## Teleportación cuántica
$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$$
Vamos a implementar el circuito de la figura, que nos servirá para teleportar el qubit $\ket{\psi}$ 

<img src="Images/teleport_circuit.png" width=80%>

In [ ]:
import projectq, random, time
from projectq.ops import Measure, H, CNOT, Z, X

#Primero creamos un qubit al azar (establecemos el valor de las
#amplitudes directamente, mediante números generados aleatoriamente)

a1 = random.random()
a2 = random.random()
b1 = random.random()
b2 = random.random()

# Hay que normalizar

norma = (a1**2 + a2**2 + b1**2 + b2**2)**0.5

a = complex(a1/norma,a2/norma) #Amplitud de |0> 
b = complex(b1/norma,b2/norma) #Amplitud de |1>

eng = projectq.MainEngine()

psi = eng.allocate_qubit()
eng.flush()

eng.backend.set_wavefunction([a,b],psi)

print("El qubit que tiene Alice es:")
print(eng.backend.get_amplitude("0",psi),"|0> + ",
      eng.backend.get_amplitude("1",psi),"|1>")

# Ahora creamos el par entrelazado que van a compartir Alice y Bob

bell = eng.allocate_qureg(2)
H | bell[0]
CNOT | (bell[0],bell[1])

# Aplicamos la CNOT a |psi> y a la parte de Alice del par de Bell
# También aplicamos la puerta H

CNOT | (psi,bell[0])
H | psi

# Medimos los qubits de Alice y guardamos el resultado

Measure | psi 
Measure | bell[0]

a = int(bell[0])
b = int(psi)

print("")
print("Teleporting", end='')
for _ in range(10):
    time.sleep(.4)
    print(".",end='')
print("")
print("")

# Alice comunica sus medidas a Bob y este aplica X y Z en consecuencia

if(a==1):
    X | bell[1]
if(b==1):
    Z | bell[1]
    
# Accedemos a las amplitudes para ver el qubit que tiene ahora Bob

eng.flush()

str0 = str(b)+str(a)+"0"
str1 = str(b)+str(a)+"1"

print("El qubit que ahora tiene Bob es:")
print(eng.backend.get_amplitude(str0,psi+bell),"|0> + ",
      eng.backend.get_amplitude(str1,psi+bell),"|1>")

Measure | bell[1]



## Codificación superdensa

Ahora, vamos a implementar el circuito de la codificación superdensa, que nos permite enviar dos bits clásicos mediante un único qubit, siempre que tengamos un par de bell compartido previamente

<img src="Images/superdense.png" width=100%>

In [ ]:
from projectq.ops import All

# Comenzamos creando el par entrelazado compartido por Alice y Bob

eng=projectq.MainEngine()

bell = eng.allocate_qureg(2)
H | bell[0]
CNOT | (bell[0],bell[1])

# Alice decide los bits que quiere enviar

b1 = random.randint(0,1)
b2 = random.randint(0,1)

print("Alice quiere enviar ",b1,b2)

# Y aplica las puertas correspondientes

if(b2==1):
    X | bell[0]
if(b1==1):
    Z | bell[0]
    
# Envía su parte del qubit a Bob

print("")
print("Sending", end='')
for _ in range(10):
    time.sleep(.2)
    print(".",end='')
print("")
print("")

# Bob procede a aplicar sus puertas y a medir

CNOT | (bell[0],bell[1])
H | bell[0]

All(Measure) | bell

print("Bob ha recibido ",int(bell[0]),int(bell[1]))
